# Modules

In [1]:
import json
import os
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup as bs

# Constantes

In [2]:
PATH_DATA = r"C:\Users\ahoun\Documents\Documents\Nuku Data\ETL-real-estate-prices\Data"
PATH_DATA_RAW = rf"{PATH_DATA}\Raw"

# Application

In [3]:
if os.path.exists(PATH_DATA):
    print('Le dossier Data existe déja.')
else: 
    os.mkdir(PATH_DATA)
    print('Le dossier a été créé avec succès.')

Le dossier Data existe déja.


In [4]:
if os.path.exists(PATH_DATA_RAW):
    print('Le dossier Data raw existe déja.')
else: 
    os.mkdir(PATH_DATA_RAW)
    print('Le dossier Data Raw a été créé avec succès.')

Le dossier Data raw existe déja.


## Téléchargement données d'une page

In [5]:
url = "https://www.immoscout24.ch/en/house/buy/city-bern?pn=1&r=40"
response = requests.get(url)

In [6]:
html = response.content

In [7]:
soup = bs(html, "html")

In [8]:
descs = soup.find_all('div', {'class': "Box-cYFBPY Flex-feqWzG MetaBody-iCkzuU JVKAR dCDRxm dUKbXG"})

data = []
    

for i,des in enumerate(descs):
    content = {'id': i,
                'title': None,
                'description': None,
                'room': None,
                'surface': None,
                'price': None
                }
    content['title'] = des.find('h2').get_text()
    
    if des.find('p'):
        content['description'] = des.find('p').get_text()
    else:
        content['description'] = None


    list_reste = des.find('h3').get_text(strip=True).split('CHF')
    if len(list_reste) == 2:
        price = list_reste[1]
        price = price.replace('CHF ', '')
        price = price.replace('.—', '')
        price = price.replace(',', '').strip()
    else:
        price = None
        

    number_room = list_reste[0].split(',')[0]
    number_room = number_room.replace('rooms', '')
    
    surface = list_reste[0].split(',')[1]
    surface = surface.replace('m²', '')
    
    
    content['room'] = number_room.strip()
    content['surface'] = surface.strip()
    content['price'] = price
    data.append(content)

data

[{'id': 0,
  'title': '«Trop tard déjà réservé (Villa F)»',
  'description': "Nouvelle promotion de villas dans un magnifique quartier ! Érigé sur une jolie parcelle à quelques pas des transports publics et de toutes les commodités, ce bien saura vous charmer par son architecture et ses grands espaces ! Le choix des finitions et des aménagements est au gré des acquéreurs.Dans la partie jour, vous découvrirez un grand hall avec armoires murales et une spacieuse et lumineuse pièce de vie. Elle sera pourvue d'un séjour et d'un espace dînatoire communiquant avec une grande cuisine à l'américaine. Cette dernière sera entièrem",
  'room': '6.5',
  'surface': '210',
  'price': '1482480'},
 {'id': 1,
  'title': '«Et si le rêve commençait ici (Villa B)»',
  'description': "Nouvelle promotion de villas dans un magnifique quartier ! Érigé sur une jolie parcelle à quelques pas des transports publics et de toutes les commodités, ce bien saura vous charmer par son architecture et ses grands espaces 

In [9]:
# Nom du fichier de sortie
page = 1
page_path = rf"{PATH_DATA_RAW}/Page_{page}"
if os.path.exists(page_path):
    print(f'Le dossier pour la page {page} existe déja.')
else: 
    os.mkdir(page_path)
    print(f'Le dossier pour la page {page} a été créé avec succès.')
    

nom_fichier = rf"{page_path}/data_page_{page}.json"

# Sauvegarder le dictionnaire en JSON
with open(nom_fichier, "w", encoding="utf-8") as fichier:
    json.dump(data, fichier, ensure_ascii=False)

Le dossier pour la page 1 existe déja.


In [10]:
df = pd.read_json(nom_fichier)
df

,id,title,description,room,surface,price
0,0,«Trop tard déjà réservé (Villa F)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,210,1482480.0
1,1,«Et si le rêve commençait ici (Villa B)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,230,1482480.0
2,2,«Détente et bien-être garantis (Villa A)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,230,1520820.0
3,3,«Votre rêve à portée de main (Villa G)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,220,1482480.0
4,4,«Idéal pour un nouveau départ (Villa C)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,230,1482480.0
5,5,«Le bien-être au quotidien (Villa E)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,230,1520820.0
6,6,«Einfamilienhaus mit herrlicher Aussicht»,"In Hofstetten SO, an herrlicher Aussichtslage ...",5.5,300,2370690.0
7,7,«Modernes Einfamilienhaus mit unverbaubarer We...,Modernes Einfamilienhaus mit unverbaubarer Wei...,7.5,240,NaN
8,8,«Exklusive Doppeleinfamilienhäuser mit traumha...,Exklusive Doppeleinfamilienhäuser mit traumhaf...,5.5,260,1639670.0
9,9,"«Obstgarte, moderns 3.5-Zemmer-Egg-Eifamiliehuus»",In der idyllischen Vorortsgemeinde von Zofinge...,5.5,190,1520820.0


## Téléchargement données de plusieurs pages

In [11]:
page = 1
url = f"https://www.immoscout24.ch/en/house/buy/city-bern?r=40"
response = requests.get(url)


while response.status_code == 200 :
    url = f"https://www.immoscout24.ch/en/house/buy/city-bern?pn={page}&r=40"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "html")
    descs = soup.find_all('div', {'class': "Box-cYFBPY Flex-feqWzG MetaBody-iCkzuU JVKAR dCDRxm dUKbXG"})
    data = []
    
    for i,des in enumerate(descs):
        content = {'id': i,
                    'title': None,
                    'description': None,
                    'room': None,
                    'surface': None,
                    'price': None
                    }
        content['title'] = des.find('h2').get_text()
    
        if des.find('p'):
            content['description'] = des.find('p').get_text()
        else:
            content['description'] = None

        list_reste = des.find('h3').get_text(strip=True).split('CHF')
        if len(list_reste) == 2:
            price = list_reste[1]
            price = price.replace('CHF ', '')
            price = price.replace('.—', '')
            price = price.replace(',', '').strip()
        else:
            price = None
            
        
        list_rooms = list_reste[0].split('rooms')
        if len(list_rooms) == 2:
            liste_room = des.find('h3').get_text(strip=True).split('rooms')
            number_room = liste_room[0].replace('rooms','').strip()
            list_for_surface = list_rooms[1].split('m²')
            surface = list_for_surface[0].replace(',','').strip()
        else:
            if "rooms" in list_rooms:
                number_room = list_rooms.replace('rooms','').strip()       
        
        content['room'] = number_room.strip()
        content['surface'] = surface.strip()
        content['price'] = price
        data.append(content)
        
        
    # Nom du fichier de sortie
    page_path = rf"{PATH_DATA_RAW}/Page_{page}"
    if os.path.exists(page_path):
        print(f'Le dossier pour la page {page} existe déja.')
    else: 
        os.mkdir(page_path)
        print(f'Le dossier pour la page {page} a été créé avec succès.')
        

    nom_fichier = rf"{page_path}/data_page_{page}.json"

    # Sauvegarder le dictionnaire en JSON
    with open(nom_fichier, "w", encoding="utf-8") as fichier:
        json.dump(data, fichier, ensure_ascii=False)
        fichier.close()
        
    page += 1

Le dossier pour la page 1 existe déja.
Le dossier pour la page 2 existe déja.
Le dossier pour la page 3 existe déja.
Le dossier pour la page 4 existe déja.
Le dossier pour la page 5 existe déja.
Le dossier pour la page 6 existe déja.
Le dossier pour la page 7 existe déja.
Le dossier pour la page 8 existe déja.
Le dossier pour la page 9 existe déja.
Le dossier pour la page 10 existe déja.
Le dossier pour la page 11 existe déja.
Le dossier pour la page 12 existe déja.
Le dossier pour la page 13 existe déja.
Le dossier pour la page 14 existe déja.
Le dossier pour la page 15 existe déja.
Le dossier pour la page 16 existe déja.
Le dossier pour la page 17 existe déja.
Le dossier pour la page 18 existe déja.
Le dossier pour la page 19 existe déja.
Le dossier pour la page 20 existe déja.
Le dossier pour la page 21 existe déja.
Le dossier pour la page 22 existe déja.
Le dossier pour la page 23 existe déja.
Le dossier pour la page 24 existe déja.
Le dossier pour la page 25 existe déja.
Le dossie

In [12]:
path_page = rf'{PATH_DATA_RAW}\Page_1\data_page_1.json'
df = pd.read_json(path_page)
df

,id,title,description,room,surface,price
0,0,«Trop tard déjà réservé (Villa F)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,210,1482480.0
1,1,«Et si le rêve commençait ici (Villa B)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,230,1482480.0
2,2,«Détente et bien-être garantis (Villa A)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,230,1520820.0
3,3,«Votre rêve à portée de main (Villa G)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,220,1482480.0
4,4,«Idéal pour un nouveau départ (Villa C)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,230,1482480.0
5,5,«Le bien-être au quotidien (Villa E)»,Nouvelle promotion de villas dans un magnifiqu...,6.5,230,1520820.0
6,6,«Einfamilienhaus mit herrlicher Aussicht»,"In Hofstetten SO, an herrlicher Aussichtslage ...",5.5,300,2370690.0
7,7,«Modernes Einfamilienhaus mit unverbaubarer We...,Modernes Einfamilienhaus mit unverbaubarer Wei...,7.5,240,NaN
8,8,«Exklusive Doppeleinfamilienhäuser mit traumha...,Exklusive Doppeleinfamilienhäuser mit traumhaf...,5.5,260,1639670.0
9,9,"«Obstgarte, moderns 3.5-Zemmer-Egg-Eifamiliehuus»",In der idyllischen Vorortsgemeinde von Zofinge...,5.5,190,1520820.0
